In [1]:
from bs4 import BeautifulSoup
import requests

url='https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
response = requests.get(url)
page = response.text

soup = BeautifulSoup(page,"lxml")
url_tail=[]
airline_data=[]
airlines_name=[]

for group in soup.find_all('div', class_="a_z_col_group"):
    for each in group.find_all('li'):
        url_tail.append(each.find('a').get('href'))
        airlines_name.append(each.text)
        
base='https://www.airlinequality.com/'

for url in url_tail:
    responses = requests.get(base+url)
    pages = responses.text

    dish = BeautifulSoup(pages,"lxml")
    table=dish.find('table')
    airline_dict={}
    airline_name=airlines_name[url_tail.index(url)]
    airline_dict['Airline_Name']=airline_name
    for row in table.find_all('tr'):
        name,stars = row.find_all('td')
        airline_dict[name.text.replace(' ','_')]=int(len(stars.find_all(class_='star fill')))
    airline_dict['Review_Count']=int(dish.find('div', class_="review-count").find('span').text.strip())
    airline_dict['Rating']=int(dish.find('div', class_="rating-10 rating-large").find('span').text.strip())
    airline_data.append(airline_dict)
    
import pandas as pd
data=pd.DataFrame(airline_data)
data.set_index('Airline_Name',inplace=True)

In [2]:
data.head()

,Food_&_Beverages,Inflight_Entertainment,Seat_Comfort,Staff_Service,Value_for_Money,Review_Count,Rating
Airline_Name,,,,,,,
AB Aviation,2,0,3,3,2,3,4
Adria Airways,3,2,3,3,3,91,6
Aegean Airlines,4,2,4,4,4,605,8
Aer Lingus,3,2,3,4,3,789,6
Aero VIP,0,0,4,4,4,3,9


In [3]:
data.to_csv('data.csv')